Goal: Violin plots of the following data

Y-axis (1 plot for each variable): Weight, Bending Stiffness (D11) above "weak layer"
    Weak Layer is: 
    - Failure layer of test (ECT, PST)
    - failure layer of avalanche (Crown, flank)
    - or identified layer of concern (Layer of concern)

X-axis: Different groups of pits
    - ECTP
    - ECTN
    - PST < 0.5
    - Pits on crowns
    - Pits on flanks
    - Layer of concern


Larger dataset
- Last five snow years


Need to code

Weight
- Density from hand hardness and grain form
- Weight above weak layer (from layer densities and thickensses)

Bending stiffness
- Elastic modulus (p 1481)
    

In [9]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from snowpylot import caaml_parser

parse_pits function: Parses all pits in a specified folder

In [10]:
def parse_pits(folder_path):
    """
    Function to parse CAAML files in the specified folder
    """

    files = [
        f for f in os.listdir(folder_path) if f.endswith(".xml")
    ]  # List of all .xml files in the folder

    pits_list = []

    for file in files:  # iterate through each file in the folder
        file_path = folder_path + "/" + file  # create the file path
        pit = caaml_parser(file_path)  # parse the file
        pits_list.append(pit)

    return pits_list

Specify folders for 2020-2024 Water Years and parse files

In [11]:
# Define folders and parse pits

pits_19_20 = parse_pits("../snowpits/by_season/2019-2020")
pits_20_21 = parse_pits("../snowpits/by_season/2020-2021")
pits_21_22 = parse_pits("../snowpits/by_season/2021-2022")
pits_22_23 = parse_pits("../snowpits/by_season/2022-2023")
pits_23_24 = parse_pits("../snowpits/by_season/2023-2024")

all_pits = (
    pits_19_20 + pits_20_21 + pits_21_22 + pits_22_23 + pits_23_24
)  # list of all pits

Geldsetzer table of density from hand hardness and grain form

In [12]:
geldsetzer_df = pd.read_csv('geldsetzer_table.csv', index_col=0)

def get_density(hand_hardness, grain_form, df=geldsetzer_df):
    """
    Get density value for a specific hand hardness and grain form combination.
    
    Parameters:
    df (pd.DataFrame): The Geldsetzer table DataFrame
    hand_hardness (str): Hand hardness value (e.g., 'F-', '4F+', 'P-', etc.)
    grain_form (str): Grain form (e.g., 'PP', 'DF', 'RG', 'FC', 'DH', etc.)
    
    Returns:
    float: Density value, or NaN if not available
    """
    try:
        return df.loc[hand_hardness, grain_form]
    except KeyError as e:
        print(f"Key not found: {e}")
        return np.nan

In [13]:
d = get_density('F-', 'PP')
print(d)

69.0


Get weight from layer and density info

NOTE: What to do about
- Layers with grain forms not in Geldsetzer table?
    - Opportunity to add to the table?
- Pits where not all of the layers above the weak layer have the info needed to calculate density

In [31]:
pit = all_pits[1]
layers = pit.snow_profile.layers

for i, layer in enumerate(layers):
    print(f"Layer {i+1}:")
    # If layer has info needed to calculate density
    if layer.hardness is not None and layer.grain_form_primary is not None:
        print("Hardness:", layer.hardness, "Grain form:", layer.grain_form_primary.grain_form)
        
        density = get_density(layer.hardness, layer.grain_form_primary.grain_form)
        print("Density:", density)

        thickness_m = (layer.thickness[0])/100 # convert cm to m
        weight = thickness_m * density # Weight in kg (assuming 1 m^2 cross-section)
        print("Weight:", weight)
        print("")
    else:
        print("No density info for this layer")




Layer 1:
Hardness: F+ Grain form: PP
Density: 93.0
Weight: 10.23

Layer 2:
Hardness: I Grain form: IF
Key not found: 'IF'
Density: nan
Weight: nan

Layer 3:
Hardness: 4F+ Grain form: DF
Density: 149.0
Weight: 20.860000000000003

Layer 4:
No density info for this layer
Layer 5:
Hardness: P Grain form: RG
Density: 273.0
Weight: 125.58000000000001

